In [1]:
import pandas as pd
import requests
import time
import xml.etree.ElementTree as ET
import plotly.express as px
import calendar
import folium

# **Obtençao dos dados**

In [2]:
# Definindo os parâmetros
subbacia_cod = 39
bacia_cod = 3

url = 'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroInventario'
params = {
    'codEstDE': '',
    'codEstATE': '',
    'tpEst': '2',  # Estações hidrométricas
    'nmEst': '',
    'nmRio': '',
    'codSubBacia': subbacia_cod,
    'codBacia': '',
    'nmMunicipio': '',
    'nmEstado': '',
    'sgResp': '',
    'sgOper': '',
    'telemetrica': ''
}

# Fazendo a requisição
response = requests.get(url, params=params)

# Verificando se deu certo
if response.status_code == 200:
    xml = response.text

    # Lendo XML e transformando em DataFrame
    df_estacoes = pd.read_xml(xml,
                              xpath="//Table",
                              namespaces={"xs": "http://www.w3.org/2001/XMLSchema"})

    #Extraindo os dados apenas da ANA
    df_estacoes_ana = df_estacoes[df_estacoes['ResponsavelSigla']=="ANA"]
    codigo_municipios_ana = df_estacoes_ana['Codigo'].unique().tolist()

    # Exibindo os dados
    display(df_estacoes)

else:
    print(f"Erro na requisição: {response.status_code}")


<ipython-input-2-b270b15f30cd>:29: FutureWarning: Passing literal xml to 'read_xml' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_estacoes = pd.read_xml(xml,


,id,rowOrder,BaciaCodigo,SubBaciaCodigo,RioCodigo,RioNome,EstadoCodigo,nmEstado,MunicipioCodigo,nmMunicipio,...,TipoRedeCaptacao,TipoRedeSedimentos,TipoRedeQualAgua,TipoRedeClasseVazao,UltimaAtualizacao,Operando,Descricao,NumImagens,DataIns,DataAlt
0,Table1,0,3,39,NaN,NaN,12,PERNAMBUCO,12054500,FERNANDO DE NORONHA,...,0,0,0,0,2025-01-26 00:00:00,1,TELEMÉTRICA,0,2025-01-27 00:00:00,None
1,Table2,1,3,39,NaN,NaN,11,PARAÍBA,11006000,ALHANDRA,...,7,0,0,0,2005-06-20 00:00:00,0,None,0,None,None
2,Table3,2,3,39,NaN,NaN,11,PARAÍBA,11006000,ALHANDRA,...,7,0,0,0,2025-05-19 15:08:00,1,None,0,None,2025-05-19 15:08:00
3,Table4,3,3,39,NaN,NaN,12,PERNAMBUCO,12062000,GOIANA,...,3,0,3,0,2005-06-20 00:00:00,0,None,0,None,None
4,Table5,4,3,39,NaN,NaN,12,PERNAMBUCO,12068000,IGARASSU,...,3,0,3,0,2006-03-14 00:00:00,0,None,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780,Table781,780,3,39,NaN,NaN,13,ALAGOAS,13023000,CORURIPE,...,7,0,0,0,2010-07-15 00:00:00,1,TELEMÉTRICA,0,None,2010-07-15 00:00:00
781,Table782,781,3,39,NaN,NaN,13,ALAGOAS,13023000,CORURIPE,...,3,0,0,0,1995-05-26 00:00:00,1,None,0,None,None
782,Table783,782,3,39,NaN,NaN,13,ALAGOAS,13023000,CORURIPE,...,7,0,0,0,2022-01-05 00:00:00,1,None,0,None,2022-01-05 00:00:00
783,Table784,783,3,39,NaN,NaN,13,ALAGOAS,13023000,CORURIPE,...,7,0,0,0,2025-05-15 14:51:00,0,None,0,2010-07-12 00:00:00,2025-05-15 14:51:00


In [3]:
# Criar um DataFrame vazio para armazenar os resultados
df_chuva_total = pd.DataFrame()

# Percorrer cada estação e obter os dados de chuva
for cod_est in codigo_municipios_ana:
    url = "http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica"

    params = {
        'codEstacao': cod_est,  # Código da estação
        'dataInicio': '',  # Campo vazio para obter todas as datas disponíveis
        'dataFim': '',  # Campo vazio para obter todas as datas disponíveis
        'tipoDados': '2',  # Tipo 2 corresponde a precipitação (chuva)
        'nivelConsistencia': ''  # Campo vazio para obter todos os níveis de consistência
    }

    # Faz a requisição GET para obter os dados
    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(f"Erro ao acessar API para a estação {cod_est}. Código {response.status_code}")
        continue

    xml = response.text

    try:
        # Parse do XML
        root = ET.fromstring(xml)

        # Definir namespaces (caso necessário)
        namespaces = {
            'diffgr': 'urn:schemas-microsoft-com:xml-diffgram-v1',
            'msdata': 'urn:schemas-microsoft-com:xml-msdata'
        }

        # Lista para armazenar os dados
        dados = []

        for serie in root.findall('.//diffgr:diffgram/DocumentElement/SerieHistorica', namespaces):
            registro = {
                'EstacaoCodigo': cod_est,
                'DataHora': serie.find('DataHora').text
            }

            # Capturar chuvas e status de 1 a 31
            for dia in range(1, 32):
                chuva_tag = f'Chuva{dia:02d}'
                status_tag = f'Chuva{dia:02d}Status'

                chuva = serie.find(chuva_tag)
                status = serie.find(status_tag)

                registro[chuva_tag] = chuva.text if chuva is not None else None
                registro[status_tag] = status.text if status is not None else None

            dados.append(registro)

        # Criar DataFrame
        df_chuva = pd.DataFrame(dados)

        # Verifica se há dados e concatena com o DataFrame geral
        if not df_chuva.empty:
            df_chuva_total = pd.concat([df_chuva_total, df_chuva], ignore_index=True)
        else:
            print(f"Nenhum dado encontrado para a estação {cod_est}.")

    except ET.ParseError:
        print(f"Erro ao processar XML para a estação {cod_est}.")

    # Pausa para evitar sobrecarga na API
    time.sleep(1)

# Exibe os dados coletados
print(df_chuva_total)


Nenhum dado encontrado para a estação 734076.
Nenhum dado encontrado para a estação 735170.
Nenhum dado encontrado para a estação 735171.
Nenhum dado encontrado para a estação 835144.
Nenhum dado encontrado para a estação 835146.
Nenhum dado encontrado para a estação 835148.
Nenhum dado encontrado para a estação 835149.
Nenhum dado encontrado para a estação 835150.
Nenhum dado encontrado para a estação 835151.
Nenhum dado encontrado para a estação 835152.
Nenhum dado encontrado para a estação 835153.
Nenhum dado encontrado para a estação 835155.
Nenhum dado encontrado para a estação 835210.
Nenhum dado encontrado para a estação 835211.
Nenhum dado encontrado para a estação 836096.
Nenhum dado encontrado para a estação 836097.
Nenhum dado encontrado para a estação 936117.
Nenhum dado encontrado para a estação 936120.
Nenhum dado encontrado para a estação 936125.
Nenhum dado encontrado para a estação 936126.
Nenhum dado encontrado para a estação 936127.
Nenhum dado encontrado para a esta

In [4]:
 df_chuva_total

,EstacaoCodigo,DataHora,Chuva01,Chuva01Status,Chuva02,Chuva02Status,Chuva03,Chuva03Status,Chuva04,Chuva04Status,...,Chuva27,Chuva27Status,Chuva28,Chuva28Status,Chuva29,Chuva29Status,Chuva30,Chuva30Status,Chuva31,Chuva31Status
0,735050,2025-02-01 00:00:00,0,1,9.5,1,0,1,21.1,1,...,0,1,0,1,None,0,None,0,None,0
1,735050,2025-01-01 00:00:00,0,1,0,1,0,1,0,1,...,37.6,1,41.9,1,0,1,48.5,1,0,1
2,735050,2024-12-01 00:00:00,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,1
3,735050,2024-11-01 00:00:00,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,None,0
4,735050,2024-10-01 00:00:00,0,1,0,1,0,1,0,1,...,0,1,0,1,7.3,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28448,1036062,1989-10-01 00:00:00,0,1,0,1,1.4,1,0,1,...,0,1,0,1,0,1,0,1,0,1
28449,1036062,1989-09-01 00:00:00,3.1,1,0,1,0,1,0.3,1,...,2.4,1,0.6,1,0.2,1,0,1,None,0
28450,1036062,1989-09-01 00:00:00,3.1,1,0,1,0,1,0.3,1,...,2.4,1,0.6,1,0.2,1,0,1,None,0
28451,1036062,1989-08-01 00:00:00,None,0,None,0,None,0,None,0,...,0.1,1,3,1,6.1,1,20.6,1,7.3,1


In [5]:
len(df_chuva_total['EstacaoCodigo'].unique())

40

# **Pre-processamento**

In [6]:
df_chuva_total_copia=df_chuva_total.copy()
df_chuva_total_copia

,EstacaoCodigo,DataHora,Chuva01,Chuva01Status,Chuva02,Chuva02Status,Chuva03,Chuva03Status,Chuva04,Chuva04Status,...,Chuva27,Chuva27Status,Chuva28,Chuva28Status,Chuva29,Chuva29Status,Chuva30,Chuva30Status,Chuva31,Chuva31Status
0,735050,2025-02-01 00:00:00,0,1,9.5,1,0,1,21.1,1,...,0,1,0,1,None,0,None,0,None,0
1,735050,2025-01-01 00:00:00,0,1,0,1,0,1,0,1,...,37.6,1,41.9,1,0,1,48.5,1,0,1
2,735050,2024-12-01 00:00:00,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,1
3,735050,2024-11-01 00:00:00,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,None,0
4,735050,2024-10-01 00:00:00,0,1,0,1,0,1,0,1,...,0,1,0,1,7.3,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28448,1036062,1989-10-01 00:00:00,0,1,0,1,1.4,1,0,1,...,0,1,0,1,0,1,0,1,0,1
28449,1036062,1989-09-01 00:00:00,3.1,1,0,1,0,1,0.3,1,...,2.4,1,0.6,1,0.2,1,0,1,None,0
28450,1036062,1989-09-01 00:00:00,3.1,1,0,1,0,1,0.3,1,...,2.4,1,0.6,1,0.2,1,0,1,None,0
28451,1036062,1989-08-01 00:00:00,None,0,None,0,None,0,None,0,...,0.1,1,3,1,6.1,1,20.6,1,7.3,1


In [7]:
df_chuva_total_copia.dtypes

,0
EstacaoCodigo,int64
DataHora,object
Chuva01,object
Chuva01Status,object
Chuva02,object
...,...
Chuva29Status,object
Chuva30,object
Chuva30Status,object
Chuva31,object


In [8]:
# Coloando a coluna datahora em datetime
df_chuva_total_copia['DataHora'] = pd.to_datetime(df_chuva_total_copia['DataHora'])

#transformando todos os dados que estao como "object" em "float"
for i in df_chuva_total_copia.columns:
  if(df_chuva_total_copia[i].dtype=='object'):
    df_chuva_total_copia[i]=df_chuva_total_copia[i].str.replace(',','.').astype(float)
df_chuva_total_copia

,EstacaoCodigo,DataHora,Chuva01,Chuva01Status,Chuva02,Chuva02Status,Chuva03,Chuva03Status,Chuva04,Chuva04Status,...,Chuva27,Chuva27Status,Chuva28,Chuva28Status,Chuva29,Chuva29Status,Chuva30,Chuva30Status,Chuva31,Chuva31Status
0,735050,2025-02-01,0.0,1.0,9.5,1.0,0.0,1.0,21.1,1.0,...,0.0,1.0,0.0,1.0,NaN,0.0,NaN,0.0,NaN,0.0
1,735050,2025-01-01,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,37.6,1.0,41.9,1.0,0.0,1.0,48.5,1.0,0.0,1.0
2,735050,2024-12-01,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
3,735050,2024-11-01,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,NaN,0.0
4,735050,2024-10-01,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,7.3,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28448,1036062,1989-10-01,0.0,1.0,0.0,1.0,1.4,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
28449,1036062,1989-09-01,3.1,1.0,0.0,1.0,0.0,1.0,0.3,1.0,...,2.4,1.0,0.6,1.0,0.2,1.0,0.0,1.0,NaN,0.0
28450,1036062,1989-09-01,3.1,1.0,0.0,1.0,0.0,1.0,0.3,1.0,...,2.4,1.0,0.6,1.0,0.2,1.0,0.0,1.0,NaN,0.0
28451,1036062,1989-08-01,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.1,1.0,3.0,1.0,6.1,1.0,20.6,1.0,7.3,1.0


In [9]:
df_chuva_total_copia.dtypes

,0
EstacaoCodigo,int64
DataHora,datetime64[ns]
Chuva01,float64
Chuva01Status,float64
Chuva02,float64
...,...
Chuva29Status,float64
Chuva30,float64
Chuva30Status,float64
Chuva31,float64


In [10]:
#filtrando as colunas

colunas_chuva = []
colunas_chuva.append('EstacaoCodigo')
colunas_chuva.append('DataHora')
for coluna in df_chuva_total_copia.columns:
    if coluna.startswith('Chuva') and 'Status' not in coluna:   #funçao baseada em strings que filtra as colunas que começam com chuva e nao tem status.
        colunas_chuva.append(coluna)

df_copia_filtrado = df_chuva_total_copia[colunas_chuva]
df_copia_filtrado

,EstacaoCodigo,DataHora,Chuva01,Chuva02,Chuva03,Chuva04,Chuva05,Chuva06,Chuva07,Chuva08,...,Chuva22,Chuva23,Chuva24,Chuva25,Chuva26,Chuva27,Chuva28,Chuva29,Chuva30,Chuva31
0,735050,2025-02-01,0.0,9.5,0.0,21.1,65.7,55.6,3.7,0.0,...,0.0,0.0,1.7,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,735050,2025-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.4,0.0,1.8,1.6,37.6,41.9,0.0,48.5,0.0
2,735050,2024-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,735050,2024-11-01,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.6,...,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,735050,2024-10-01,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,...,0.0,0.0,0.4,0.3,0.0,0.0,0.0,7.3,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28448,1036062,1989-10-01,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,...,45.7,0.1,0.0,0.4,2.9,0.0,0.0,0.0,0.0,0.0
28449,1036062,1989-09-01,3.1,0.0,0.0,0.3,1.7,1.9,1.3,0.8,...,5.0,13.8,1.5,4.4,0.0,2.4,0.6,0.2,0.0,NaN
28450,1036062,1989-09-01,3.1,0.0,0.0,0.3,1.7,1.9,1.3,0.8,...,5.0,13.8,1.5,4.4,0.0,2.4,0.6,0.2,0.0,NaN
28451,1036062,1989-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.6,6.5,5.9,13.5,0.7,0.1,3.0,6.1,20.6,7.3


In [11]:
# Arrumando o df para análises
resultados = []

for _, linha in df_copia_filtrado.iterrows():  # Pegando os períodos exatos de tempo com o dia certo
    ano, mes = linha['DataHora'].year, linha['DataHora'].month
    _, periodo = calendar.monthrange(ano, mes)  # Último dia do mês
    dias = pd.date_range(start=f'{ano}-{mes:02d}-01', periods=periodo, freq='D')
    # Coletando os valores de chuva para os dias do mês
    valores = [linha[f'Chuva{dia:02d}'] for dia in range(1, periodo + 1)]

    # Montando os resultados com zip
    for data, valor in zip(dias, valores):
        resultados.append({'EstacaoCodigo': linha['EstacaoCodigo'],'Data': data,'Valor': valor})

df_final = pd.DataFrame(resultados)
df_final

,EstacaoCodigo,Data,Valor
0,735050,2025-02-01,0.0
1,735050,2025-02-02,9.5
2,735050,2025-02-03,0.0
3,735050,2025-02-04,21.1
4,735050,2025-02-05,65.7
...,...,...,...
866007,1036062,1989-08-27,0.1
866008,1036062,1989-08-28,3.0
866009,1036062,1989-08-29,6.1
866010,1036062,1989-08-30,20.6


# **Gantt**

In [12]:
analise_gantt = df_final.copy()
analise_gantt

,EstacaoCodigo,Data,Valor
0,735050,2025-02-01,0.0
1,735050,2025-02-02,9.5
2,735050,2025-02-03,0.0
3,735050,2025-02-04,21.1
4,735050,2025-02-05,65.7
...,...,...,...
866007,1036062,1989-08-27,0.1
866008,1036062,1989-08-28,3.0
866009,1036062,1989-08-29,6.1
866010,1036062,1989-08-30,20.6


In [13]:
analise_gantt.isnull().sum()

,0
EstacaoCodigo,0
Data,0
Valor,19489


In [14]:
analise_gantt

,EstacaoCodigo,Data,Valor
0,735050,2025-02-01,0.0
1,735050,2025-02-02,9.5
2,735050,2025-02-03,0.0
3,735050,2025-02-04,21.1
4,735050,2025-02-05,65.7
...,...,...,...
866007,1036062,1989-08-27,0.1
866008,1036062,1989-08-28,3.0
866009,1036062,1989-08-29,6.1
866010,1036062,1989-08-30,20.6


In [15]:
# Converter código para string (evita escala numérica no eixo Y)
analise_gantt['EstacaoCodigo'] = analise_gantt['EstacaoCodigo'].astype(str)

#ordenando os valores para ficar mais organizado
analise_gantt = analise_gantt.sort_values(['EstacaoCodigo', 'Data'])

# Diferença de tempo entre registros consecutivos
analise_gantt['Delta'] = analise_gantt.groupby('EstacaoCodigo')['Data'].diff()

# Define uma quebra após 1 dia de ausência
limite_quebra = pd.Timedelta(days=1)
analise_gantt['Nova_Sessao'] = (analise_gantt['Delta'] > limite_quebra).fillna(True)

# Numera os períodos contínuos por estação
analise_gantt['Grupo'] = analise_gantt.groupby('EstacaoCodigo')['Nova_Sessao'].cumsum()

# Agrupa para encontrar início e fim de cada período contínuo
intervalos = analise_gantt.groupby(['EstacaoCodigo', 'Grupo'])['Data'].agg(Inicio='min', Fim='max').reset_index()

# Gera o gráfico
fig = px.timeline(
    intervalos,
    x_start="Inicio",
    x_end="Fim",
    y="EstacaoCodigo",)

# Layout do gráfico
fig.update_layout(
    title="Disponibilidade de Dados por Estação",
    height=25 * intervalos['EstacaoCodigo'].nunique(),
    xaxis_title="Data (dias)",
    yaxis_title="Estação",
    showlegend=False
)

# Inverter eixo Y para melhor visualização (opcional)
fig.update_yaxes(autorange="reversed")

# Exibir
fig.show()


# **Mapa folium**

In [16]:
#Obtendo os valores de latitude e longitude para as estaçoes com dados
analise_mapa_com_dados = df_estacoes_ana[df_estacoes_ana['Codigo'].isin(df_chuva_total['EstacaoCodigo'].unique())]
#filtrando
analise_mapa = analise_mapa_com_dados[['Codigo','Latitude','Longitude']]
analise_mapa['dados'] = "Sim"
analise_mapa

<ipython-input-16-1f570dcf2fc2>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Codigo,Latitude,Longitude,dados
54,735050,-7.96810,-35.1567,Sim
65,735066,-7.89420,-35.1733,Sim
66,735067,-7.94280,-35.6342,Sim
87,735100,-7.87890,-35.4519,Sim
119,735157,-7.84280,-35.1825,Sim
120,735158,-7.85470,-35.7644,Sim
121,735159,-7.91000,-35.9886,Sim
122,735169,-7.96810,-35.1158,Sim
125,735173,-7.54280,-35.0686,Sim
182,736040,-7.98640,-36.5006,Sim


In [17]:
#Obtendo os valores de latitude e longitude para as estaçoes sem dados
analise_mapa_sem_dados = df_estacoes_ana[~df_estacoes_ana['Codigo'].isin(df_chuva_total['EstacaoCodigo'].unique())]
analise_mapa_sem_dados_filtrado = analise_mapa_sem_dados[['Codigo','Latitude','Longitude']]
analise_mapa_sem_dados_filtrado['dados']="nao"
analise_mapa_sem_dados_filtrado

<ipython-input-17-73ee18ec19bd>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Codigo,Latitude,Longitude,dados
39,734076,-7.6569,-34.3711,nao
123,735170,-7.5128,-35.3269,nao
124,735171,-7.6506,-35.3206,nao
361,835144,-8.8161,-35.9339,nao
362,835146,-8.8967,-35.7678,nao
364,835148,-8.6786,-35.7233,nao
365,835149,-8.5356,-35.8506,nao
366,835150,-8.3803,-35.4553,nao
367,835151,-8.7992,-35.2050,nao
368,835152,-8.4972,-35.4978,nao


In [18]:
analise_mapa = pd.concat([analise_mapa, analise_mapa_sem_dados_filtrado])
analise_mapa

,Codigo,Latitude,Longitude,dados
54,735050,-7.9681,-35.1567,Sim
65,735066,-7.8942,-35.1733,Sim
66,735067,-7.9428,-35.6342,Sim
87,735100,-7.8789,-35.4519,Sim
119,735157,-7.8428,-35.1825,Sim
...,...,...,...,...
744,936125,-9.6858,-36.2853,nao
745,936126,-9.7436,-36.5039,nao
746,936127,-9.1831,-36.0436,nao
757,936138,-9.1322,-36.3231,nao


In [19]:
#Mapa
# Definindo o centro inicial do mapa (ex: média das coordenadas)
lat_media = analise_mapa['Latitude'].mean()
lon_media = analise_mapa['Longitude'].mean()

# Criando o mapa com o centro e nível de zoom
mapa = folium.Map(location=[lat_media, lon_media], zoom_start=9)

# Adicionando marcadores para cada estação
for _, row in analise_mapa.iterrows():
    cor = 'blue' if row['dados'] == 'Sim' else 'red'
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Estação: {row['Codigo']}<br>tem dados: {row['dados']}",
        icon=folium.Icon(color=cor, icon='tint', prefix='fa')
    ).add_to(mapa)

mapa


# **Exportando dados para a atv3**

In [20]:
df_final

,EstacaoCodigo,Data,Valor
0,735050,2025-02-01,0.0
1,735050,2025-02-02,9.5
2,735050,2025-02-03,0.0
3,735050,2025-02-04,21.1
4,735050,2025-02-05,65.7
...,...,...,...
866007,1036062,1989-08-27,0.1
866008,1036062,1989-08-28,3.0
866009,1036062,1989-08-29,6.1
866010,1036062,1989-08-30,20.6


In [21]:
atv3= df_final[df_final['EstacaoCodigo']==936121]
atv3

,EstacaoCodigo,Data,Valor
841722,936121,2025-02-01,0.0
841723,936121,2025-02-02,0.0
841724,936121,2025-02-03,0.0
841725,936121,2025-02-04,0.0
841726,936121,2025-02-05,0.7
...,...,...,...
846739,936121,2011-06-26,0.1
846740,936121,2011-06-27,0.0
846741,936121,2011-06-28,0.0
846742,936121,2011-06-29,0.0


In [24]:
coordenadas = analise_mapa[analise_mapa['Codigo']==936121]
coordenadas

,Codigo,Latitude,Longitude,dados
740,936121,-9.3664,-36.4194,Sim


In [31]:
atv3_completo = pd.merge(atv3,coordenadas[['Codigo','Latitude','Longitude']], left_on='EstacaoCodigo', right_on='Codigo' )
atv3_completo

,EstacaoCodigo,Data,Valor,Codigo,Latitude,Longitude
0,936121,2025-02-01,0.0,936121,-9.3664,-36.4194
1,936121,2025-02-02,0.0,936121,-9.3664,-36.4194
2,936121,2025-02-03,0.0,936121,-9.3664,-36.4194
3,936121,2025-02-04,0.0,936121,-9.3664,-36.4194
4,936121,2025-02-05,0.7,936121,-9.3664,-36.4194
...,...,...,...,...,...,...
5017,936121,2011-06-26,0.1,936121,-9.3664,-36.4194
5018,936121,2011-06-27,0.0,936121,-9.3664,-36.4194
5019,936121,2011-06-28,0.0,936121,-9.3664,-36.4194
5020,936121,2011-06-29,0.0,936121,-9.3664,-36.4194


In [33]:
#exportando
#liberando o acesso aos arquivos do drive

from google.colab import drive
drive.mount('/content/drive')

caminho = '/content/drive/MyDrive/df_atv2.xlsx'
atv3_completo.to_excel(caminho, index=False)

print(f"Arquivo salvo em: {caminho}")

Mounted at /content/drive
Arquivo salvo em: /content/drive/MyDrive/df_atv2.xlsx
